# Demos: Lecture 11

In [1]:
import pennylane as qml
from pennylane import numpy as np

## Demo 1: Quantum phase estimation

<img src="fig/qpe-full-screencap.png" width="800px">

In [2]:
T = qml.T.compute_matrix()
X = qml.PauliX.compute_matrix()
I = np.eye(2)

U = np.kron(T, np.kron(X, I))

In [3]:
eigvals, eigvecs = np.linalg.eig(U)

target_eigvec = eigvecs[:, 4]
target_eigval = eigvals[4]

In [5]:
np.angle(target_eigval) / (2 * np.pi) # 001

0.125

In [4]:
target_eigvec

tensor([-0.        +0.00000000e+00j, -0.        +0.00000000e+00j,
        -0.        +0.00000000e+00j, -0.        +0.00000000e+00j,
         0.70710678+0.00000000e+00j, -0.        +0.00000000e+00j,
         0.70710678+3.25176795e-17j, -0.        +0.00000000e+00j], requires_grad=True)

In [6]:
estimation_wires = [0, 1, 2]
target_wires = [3, 4, 5]

dev = qml.device("default.qubit", wires=estimation_wires+target_wires, shots=1) 

In [25]:
@qml.qnode(dev)
def quantum_phase_estimation():
    # Step 1
    for wire in estimation_wires:
        qml.Hadamard(wires=wire)
        
    qml.MottonenStatePreparation(target_eigvec, wires=target_wires)
    
    # Step 2
    for power_of_two, control_wire in enumerate(estimation_wires[::-1]):
        qml.ControlledQubitUnitary(
            np.linalg.matrix_power(U, 2 ** power_of_two), 
            control_wires=control_wire,
            wires=target_wires
        )
        
    # Step 3
    qml.adjoint(qml.QFT)(wires=estimation_wires)
    
    # Measure
    return qml.sample(wires=estimation_wires)

In [27]:
for _ in range(20):
    print(quantum_phase_estimation())

[0 0 1]
[0 0 1]
[0 0 1]
[0 0 1]
[0 0 1]
[0 0 1]
[0 0 1]
[0 0 1]
[0 0 1]
[0 0 1]
[0 0 1]
[0 0 1]
[0 0 1]
[0 0 1]
[0 0 1]
[0 0 1]
[0 0 1]
[0 0 1]
[0 0 1]
[0 0 1]


In [21]:
@qml.qnode(dev)
def qpe_with_template():
    qml.MottonenStatePreparation(target_eigvec, wires=target_wires)
    qml.QuantumPhaseEstimation(
        U, 
        target_wires=target_wires,
        estimation_wires=estimation_wires
    )
    return qml.sample(wires=estimation_wires)

In [24]:
qpe_with_template()

tensor([0, 0, 1], requires_grad=True)

## Demo 2: Order finding

In [ ]:
from lecture11_helpers import *

In [ ]:
N = 13
a = 5

In [ ]:
for exp in range(1, N):
    if (a ** exp) % N == 1:
        print(f"The order of {a} is {exp}")
        break

In [ ]:
U_Na = get_U_Na(a, N)

num_estimation_qubits = 10
num_target_qubits = int(np.log2(len(U_Na)))

estimation_wires = range(num_estimation_qubits)
target_wires = range(num_estimation_qubits, num_estimation_qubits + num_target_qubits)

In [ ]:
dev = qml.device('default.qubit', wires=num_estimation_qubits+num_target_qubits, shots=1)

@qml.qnode(dev)
def find_order():

    return qml.sample(wires=estimation_wires)

In [ ]:
possible_r = []

for _ in range(10):
    sample = find_order()
    phase = fractional_binary_to_float(sample)
    est_r = phase_to_order(phase, N)
    possible_r.append(est_r)